In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [3]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [4]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)


In [5]:
# 111 words to fix
len(keyword_trans_dict)


111

In [6]:
def fix_broken_linkedin_files(glob_path):
    fix_list = []
    data_dict = {}
    data_list = []

    if glob_path.endswith('.json'):

        with open(glob_path, encoding = 'utf-8') as csv_file_handler:
            csv_reader = csv.DictReader(csv_file_handler)

            for rows in csv_reader:
                first_key = str(list(rows.keys())[0])
                key = rows[first_key]
                data_dict[key] = rows

        for num in data_dict:
            data_list.append(data_dict[num])

        with open(glob_path, 'w', encoding = 'utf-8') as json_file_handler:
            json_file_handler.write(json.dumps(data_list, indent = 4))
    
    return data_list


In [7]:
def fix_keywords(df_temp):
    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        for key, value in keyword_trans_dict.items():
            df_temp.loc[
                df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).apply(
                lambda x: x.lower().strip()
                ) == str(key).lower().strip(), 'Search Keyword'
            ] = str(value).lower().strip()

        unfixed = df_temp.loc[
            df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
        ]

        if len(unfixed) != 0:
            for key, value in keyword_trans_dict.items():
                for idx, row in df_temp.iterrows():
                    if row['Search Keyword'].astype(str).lower().strip() == str(key).lower().strip():
                        df_temp.loc[idx, 'Search Keyword'] = str(value).lower().strip()
    

    return df_temp


#### Read paths

In [8]:
glob_paths = []

for site in site_list:
    glob_paths.extend(glob.glob(f'{scraped_data}/{site}/Data/*.json')+glob.glob(f'{scraped_data}/{site}/Data/*.csv'))


In [9]:
# 955 json and csv files
len(glob_paths)


955

#### Use paths to open files, fix keywords, and drop unneeded columns

In [10]:
# Fix list catches all incorrect/faculty keyword search terms

fix_list = []

# Appended data catches all the fixed and cleaned dfs
appended_data = []

for glob_path in glob_paths:

    if glob_path.endswith('.json'):
        try:
            df_temp = pd.read_json(glob_path).reset_index(drop=True)
        except ValueError:
            fix_list.append(glob_path)
            if 'scraped_data/LinkedIn/Data/linkedin_jobs_df_' in glob_path:
                data_json = fix_broken_linkedin_files(glob_path)
                try:
                    df_temp = pd.read_json(glob_path).reset_index(drop=True)
                except ValueError:
                    fix_list.append(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path).reset_index(drop=True)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = fix_keywords(df_temp)
        df_temp.reset_index(drop=True, inplace=True)
        df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(
        df_temp.columns[
                df_temp.columns.str.contains(
                    'unnamed|index|level', regex=True, case=False, flags=re.I
                )
            ],
            axis='columns',
            inplace=True,
            errors='ignore',
        )
    
        if glob_path.endswith('.json'):
            df_temp.to_json(glob_path, orient='records')
        elif glob_path.endswith('.csv'):
            df_temp.to_csv(glob_path, index=False)

        appended_data.append(df_temp.reset_index(drop=True))

# # Concatonate list of dfs into one large df_jobs
# df_jobs = pd.concat(appended_data).reset_index(drop=True)

# # Save df_jobs to file
# if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
#     df_jobs.to_pickle(f'{data_dir}df_jobs_raw.pkl')
    
#     df_jobs.to_csv(f'{data_dir}df_jobs_raw.csv', index=False)
# else:
#     print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


In [11]:
# List of dfs, len = 527
len(appended_data)


527

In [12]:
# Concatonate list of dfs into one large df_jobs
df_jobs = pd.concat(appended_data).reset_index(drop=True)


In [13]:
# len = 204113
len(df_jobs)


204113

In [14]:
# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


In [15]:
# If we couldn't fix some keywords, we add them to list fix_list and write to file
if len(fix_list) != 0:
    print('Some keywords to fix!')
    with open(f'{data_dir}fix_list.txt', 'w') as f:
        json.dump(fix_list, f)


### START HERE IF SOURCING FROM DF_RAW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [16]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [17]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [18]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [19]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw.pkl').reset_index(drop=True)


In [20]:
# len = 204113
len(df_jobs)


204113

In [21]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204113 entries, 0 to 204112
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Search Keyword     204113 non-null  object 
 1   Platform           204113 non-null  object 
 2   Job ID             204113 non-null  object 
 3   Job Title          204113 non-null  object 
 4   Company Name       204103 non-null  object 
 5   Location           204113 non-null  object 
 6   Job Description    204098 non-null  object 
 7   Rating             51158 non-null   float64
 8   Employment Type    203053 non-null  object 
 9   Company URL        193659 non-null  object 
 10  Job URL            204113 non-null  object 
 11  Job Age            204113 non-null  object 
 12  Job Age Number     204113 non-null  object 
 13  Collection Date    204113 non-null  object 
 14  Data Row           152949 non-null  float64
 15  Tracking ID        152949 non-null  object 
 16  In

In [22]:
df_jobs.columns

Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [23]:
# Clean columns
df_jobs.columns = df_jobs.columns.to_series().apply(lambda x: str(x).strip())


In [24]:
# Drop NA
df_jobs.dropna(axis='index', how='all', inplace=True)
df_jobs.dropna(axis='columns', how='all', inplace=True)


In [25]:
# len = 204113
len(df_jobs)


204113

In [26]:
# Drop duplicates on subset of 'Job Description'
df_jobs.drop_duplicates(subset=['Job Description'], keep='first', ignore_index=True, inplace=True)


In [27]:
# len = 62579
len(df_jobs)


62579

In [28]:
# Remove any rows with missing 'Job Description'
df_jobs.drop(
    df_jobs[
        (df_jobs['Job Description'].isin(nan_list)) | 
        (df_jobs['Job Description'].isnull()) | 
        (df_jobs['Job Description'].isna())
    ].index, 
    axis='index',
    inplace=True,
    errors='ignore'

)


In [29]:
# len = 62577
len(df_jobs)


62577

In [30]:
# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_dropped.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_dropped.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


### START HERE IF SOURCING FROM DF_RAW_DROPPED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [31]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [32]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [33]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [34]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [35]:
# 62577
len(df_jobs)


62577

In [36]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [37]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [38]:
# 62577
len(df_jobs)


62577

In [39]:
# df_jobs['Language'].value_counts()

In [ ]:
translator = Translator()
googletrans_readtime_error = googletrans.client.httpx._client.httpcore._exceptions.ReadTimeout

if 'Language' not in df_jobs.columns:
    df_jobs['Language'] = np.nan # create Language col and fill it with nan

# try:
#     time.sleep(60)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)
# except:
#     time.sleep(3600)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)

for idx, row in df_jobs.iterrows():
    # This part ensures we don't start lang detection from index 0 if some lang detection was already done
    if len(str(row['Job Description'])) != 0:
        if type(row['Language']) == float and np.isnan(row['Language']): #if lang is nan, detect language

            try:
                print(f'Row {idx}: Translation in progress.')
#                 time.sleep(10)
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
                print(f'Row {idx}: Translation done.')
            except:# googletrans_readtime_error:

                if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
                    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')

                    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
                else:
                    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')

                print(f'Row {idx}: Sleeping for half an hour starting at {time.strftime("%I:%M:%S %p", time.localtime())}.')
                print('-'*30)
                time.sleep(1800)
                print(f'Row {idx}: Done sleeping.')
                print('-'*30)
                print(f'Row {idx}: Translation in progress.')
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
                print(f'Row {idx}: Translation done.')

        else: # elif lang is not nan, skip and go to next idx
            continue

if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')

    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


Row 20789: Translation in progress.
Row 20789: Translation done.
Row 20790: Translation in progress.
Row 20790: Translation done.
Row 20791: Translation in progress.
Row 20791: Translation done.
Row 20792: Translation in progress.
Row 20792: Translation done.
Row 20793: Translation in progress.
Row 20793: Translation done.
Row 20794: Translation in progress.
Row 20794: Translation done.
Row 20795: Translation in progress.
Row 20795: Translation done.
Row 20796: Translation in progress.
Row 20796: Translation done.
Row 20797: Translation in progress.
Row 20797: Translation done.
Row 20798: Translation in progress.
Row 20798: Translation done.
Row 20799: Translation in progress.
Row 20799: Translation done.
Row 20800: Translation in progress.
Row 20800: Translation done.
Row 20801: Translation in progress.
Row 20801: Translation done.
Row 20802: Translation in progress.
Row 20802: Translation done.
Row 20803: Translation in progress.
Row 20803: Translation done.
Row 20804: Translation in

Row 20915: Translation done.
Row 20916: Translation in progress.
Row 20916: Translation done.
Row 20917: Translation in progress.
Row 20917: Translation done.
Row 20918: Translation in progress.
Row 20918: Translation done.
Row 20919: Translation in progress.
Row 20919: Translation done.
Row 20920: Translation in progress.
Row 20920: Translation done.
Row 20921: Translation in progress.
Row 20921: Translation done.
Row 20922: Translation in progress.
Row 20922: Translation done.
Row 20923: Translation in progress.
Row 20923: Translation done.
Row 20924: Translation in progress.
Row 20924: Translation done.
Row 20925: Translation in progress.
Row 20925: Translation done.
Row 20926: Translation in progress.
Row 20926: Translation done.
Row 20927: Translation in progress.
Row 20927: Translation done.
Row 20928: Translation in progress.
Row 20928: Translation done.
Row 20929: Translation in progress.
Row 20929: Translation done.
Row 20930: Translation in progress.
Row 20930: Translation do

Row 21042: Translation done.
Row 21043: Translation in progress.
Row 21043: Translation done.
Row 21044: Translation in progress.
Row 21044: Translation done.
Row 21045: Translation in progress.
Row 21045: Translation done.
Row 21046: Translation in progress.
Row 21046: Translation done.
Row 21047: Translation in progress.
Row 21047: Translation done.
Row 21048: Translation in progress.
Row 21048: Translation done.
Row 21049: Translation in progress.
Row 21049: Translation done.
Row 21050: Translation in progress.
Row 21050: Translation done.
Row 21051: Translation in progress.
Row 21051: Translation done.
Row 21052: Translation in progress.
Row 21052: Translation done.
Row 21053: Translation in progress.
Row 21053: Translation done.
Row 21054: Translation in progress.
Row 21054: Translation done.
Row 21055: Translation in progress.
Row 21055: Translation done.
Row 21056: Translation in progress.
Row 21056: Translation done.
Row 21057: Translation in progress.
Row 21057: Translation do

Row 21169: Translation done.
Row 21170: Translation in progress.
Row 21170: Translation done.
Row 21171: Translation in progress.
Row 21171: Translation done.
Row 21172: Translation in progress.
Row 21172: Translation done.
Row 21173: Translation in progress.
Row 21173: Translation done.
Row 21174: Translation in progress.
Row 21174: Translation done.
Row 21175: Translation in progress.
Row 21175: Translation done.
Row 21176: Translation in progress.
Row 21176: Translation done.
Row 21177: Translation in progress.
Row 21177: Translation done.
Row 21178: Translation in progress.
Row 21178: Translation done.
Row 21179: Translation in progress.
Row 21179: Translation done.
Row 21180: Translation in progress.
Row 21180: Translation done.
Row 21181: Translation in progress.
Row 21181: Translation done.
Row 21182: Translation in progress.
Row 21182: Translation done.
Row 21183: Translation in progress.
Row 21183: Translation done.
Row 21184: Translation in progress.
Row 21184: Translation do

Row 21296: Translation done.
Row 21297: Translation in progress.
Row 21297: Translation done.
Row 21298: Translation in progress.
Row 21298: Translation done.
Row 21299: Translation in progress.
Row 21299: Translation done.
Row 21300: Translation in progress.
Row 21300: Translation done.
Row 21301: Translation in progress.
Row 21301: Translation done.
Row 21302: Translation in progress.
Row 21302: Translation done.
Row 21303: Translation in progress.
Row 21303: Translation done.
Row 21304: Translation in progress.
Row 21304: Translation done.
Row 21305: Translation in progress.
Row 21305: Translation done.
Row 21306: Translation in progress.
Row 21306: Translation done.
Row 21307: Translation in progress.
Row 21307: Translation done.
Row 21308: Translation in progress.
Row 21308: Translation done.
Row 21309: Translation in progress.
Row 21309: Translation done.
Row 21310: Translation in progress.
Row 21310: Translation done.
Row 21311: Translation in progress.
Row 21311: Translation do

...

Row 28036: Translation done.
Row 28037: Translation in progress.
Row 28037: Translation done.
Row 28038: Translation in progress.
Row 28038: Translation done.
Row 28039: Translation in progress.
Row 28039: Translation done.
Row 28040: Translation in progress.
Row 28040: Translation done.
Row 28041: Translation in progress.
Row 28041: Translation done.
Row 28042: Translation in progress.
Row 28042: Translation done.
Row 28043: Translation in progress.
Row 28043: Translation done.
Row 28044: Translation in progress.
Row 28044: Translation done.
Row 28045: Translation in progress.
Row 28045: Translation done.
Row 28046: Translation in progress.
Row 28046: Translation done.
Row 28047: Translation in progress.
Row 28047: Translation done.
Row 28048: Translation in progress.
Row 28048: Translation done.
Row 28049: Translation in progress.
Row 28049: Translation done.
Row 28050: Translation in progress.
Row 28050: Translation done.
Row 28051: Translation in progress.
Row 28051: Translation do

Row 28164: Translation done.
Row 28165: Translation in progress.
Row 28165: Translation done.
Row 28166: Translation in progress.
Row 28166: Translation done.
Row 28167: Translation in progress.
Row 28167: Translation done.
Row 28168: Translation in progress.
Row 28168: Translation done.
Row 28169: Translation in progress.
Row 28169: Translation done.
Row 28170: Translation in progress.
Row 28170: Translation done.
Row 28171: Translation in progress.
Row 28171: Translation done.
Row 28172: Translation in progress.
Row 28172: Translation done.
Row 28173: Translation in progress.
Row 28173: Translation done.
Row 28174: Translation in progress.
Row 28174: Translation done.
Row 28175: Translation in progress.
Row 28175: Translation done.
Row 28176: Translation in progress.
Row 28176: Translation done.
Row 28177: Translation in progress.
Row 28177: Translation done.
Row 28178: Translation in progress.
Row 28178: Translation done.
Row 28179: Translation in progress.
Row 28179: Translation do

Row 28291: Translation done.
Row 28292: Translation in progress.
Row 28292: Translation done.
Row 28293: Translation in progress.
Row 28293: Translation done.
Row 28294: Translation in progress.
Row 28294: Translation done.
Row 28295: Translation in progress.
Row 28295: Translation done.
Row 28296: Translation in progress.
Row 28296: Translation done.
Row 28297: Translation in progress.
Row 28297: Translation done.
Row 28298: Translation in progress.
Row 28298: Translation done.
Row 28299: Translation in progress.
Row 28299: Translation done.
Row 28300: Translation in progress.
Row 28300: Translation done.
Row 28301: Translation in progress.
Row 28301: Translation done.
Row 28302: Translation in progress.
Row 28302: Translation done.
Row 28303: Translation in progress.
Row 28303: Translation done.
Row 28304: Translation in progress.
Row 28304: Translation done.
Row 28305: Translation in progress.
Row 28305: Translation done.
Row 28306: Translation in progress.
Row 28306: Translation do

Row 28418: Translation done.
Row 28419: Translation in progress.
Row 28419: Translation done.
Row 28420: Translation in progress.
Row 28420: Translation done.
Row 28421: Translation in progress.
Row 28421: Translation done.
Row 28422: Translation in progress.
Row 28422: Translation done.
Row 28423: Translation in progress.
Row 28423: Translation done.
Row 28424: Translation in progress.
Row 28424: Translation done.
Row 28425: Translation in progress.
Row 28425: Translation done.
Row 28426: Translation in progress.
Row 28426: Translation done.
Row 28427: Translation in progress.
Row 28427: Translation done.
Row 28428: Translation in progress.
Row 28428: Translation done.
Row 28429: Translation in progress.
Row 28429: Translation done.
Row 28430: Translation in progress.
Row 28430: Translation done.
Row 28431: Translation in progress.
Row 28431: Translation done.
Row 28432: Translation in progress.
Row 28432: Translation done.
Row 28433: Translation in progress.
Row 28433: Translation do

Row 28545: Translation done.
Row 28546: Translation in progress.
Row 28546: Translation done.
Row 28547: Translation in progress.
Row 28547: Translation done.
Row 28548: Translation in progress.
Row 28548: Translation done.
Row 28549: Translation in progress.
Row 28549: Translation done.
Row 28550: Translation in progress.
Row 28550: Translation done.
Row 28551: Translation in progress.
Row 28551: Translation done.
Row 28552: Translation in progress.
Row 28552: Translation done.
Row 28553: Translation in progress.
Row 28553: Translation done.
Row 28554: Translation in progress.
Row 28554: Translation done.
Row 28555: Translation in progress.
Row 28555: Translation done.
Row 28556: Translation in progress.
Row 28556: Translation done.
Row 28557: Translation in progress.
Row 28557: Translation done.
Row 28558: Translation in progress.
Row 28558: Translation done.
Row 28559: Translation in progress.
Row 28559: Translation done.
Row 28560: Translation in progress.
Row 28560: Translation do

In [45]:
df_jobs['Language'].value_counts()

en              14349
nl               6424
['nl', 'en']        6
['en', 'nl']        4
de                  3
tr                  1
fr                  1
pl                  1
Name: Language, dtype: int64

In [56]:
df_jobs['Job Description'].loc[
    (df_jobs['Language'] == "['nl', 'en']") | 
    (df_jobs['Language'] == "['en', 'nl']")
]

2031     Software DeveloperLocatie Den HaagNewComplianc...
2481     Wat doe je als Cloud Engineer and Application ...
5097     Locatie Amsterdam, , Functie omschrijving Asse...
6081     We at Yara are part of a global network, colla...
6664     We at Yara are part of a global network, colla...
7848     Logistics and Warehousing Manager, Productieor...
10431    Wat doe je als Senior Electricity Market Devel...
12321    Job Title Technical Support Engineer, Job Loca...
12486    Senior Electronics engineer Senior NoordBraban...
12960    About The Company, , Our client is an ambitiou...
Name: Job Description, dtype: object

In [55]:
for job_description in df_jobs['Job Description'].loc[
    (df_jobs['Language'] == "['nl', 'en']") | 
    (df_jobs['Language'] == "['en', 'nl']")
].items():
    print(job_description)

(2031, 'Software DeveloperLocatie Den HaagNewCompliance Driving Value in healthcare')
(2481, 'Wat doe je als Cloud Engineer and Application Developer IT Software Specialist bij TenneT Develop robust, cloud native stateless applications Gather business stakeholder requirements Rapid development of minimal viable products Incremental product development in an agile project organization Create dashboards and present results to the business stakeholders Develop Docker containers and integrate them in a microservice architecture Microservice deployment and operation Implement CICD processes with high degree of test automation Early ideVoor de volledige vacature klik op de sollicitatiebutton Je wordt doorgestuurd naar een externe website')
(5097, 'Locatie Amsterdam, , Functie omschrijving Asset Analyst junior Our client is an international investment management company in commercial real estate Junior Asset Analyst Responsibilities')
(6081, 'We at Yara are part of a global network, collabora

In [ ]:
# df_jobs.drop(
#     df_jobs[
#         (df_jobs['Language'] == "['en', 'nl']") | 
#         (df_jobs['Language'] == "['nl', 'en']")
#     ].index, 
#         axis='index', 
#         inplace=True, 
#         errors='ignore'
# )


In [43]:
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')

In [ ]:
# Drop non-english job descriptions
# df_jobs.drop(
#     df_jobs[df_jobs['Language'] != str(language)].index, 
#     axis='index', 
#     inplace=True, 
#     errors='ignore'
# )


In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [ ]:
# Find and count unique search keywords
search_keywords = list(set(df_jobs['Search Keyword'].to_list()))


In [ ]:
len(search_keywords)


In [ ]:
search_keywords


In [ ]:
dutch_requirement_pattern = r'[Ll]anguage: [Dd]utch|[Dd]utch [Pp]referred|[Dd]utch [Re]quired|[Dd]utch [Ll]anguage|[Pp]roficient in [Dd]utch|[Ss]peak [Dd]utch|[Kk]now [Dd]utch'
english_requirement_pattern = r'[Ll]anguage: [Ee]nglish|[Ee]nglish [Pp]referred|[Ee]nglish [Re]quired|[Ee]nglish [Ll]anguage|[Pp]roficient in [Ee]nglish|[Ss]peak [Ee]nglish|[Kk]now [Ee]nglish'
    